<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_graph_modelling_language_using_select_speaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Controlling Agent Speaker Transitions with JSON Mode in Autogen
Introduction

In this notebook, we'll explore how to use OpenAI API JSON mode within the Autogen framework to generate very precise agent responses. 
In this example we will use JSON mode to provide values in the response that can be used to mathematically decide on the next speaker. 
Please find documentation about this feature in OpenAI  [here](https://platform.openai.com/docs/guides/text-generation/json-mode).

Benefits
- This contribution provides a easy to implement method for precise speaker transitions. See Motivation for more detailed discussion.


## Requirements
JSON mode is a feature of OpenAI API, however strong models (such as claude 3 Opus), can generate appropriate json as well.
AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```
 
In Your OAI_CONFIG_LIST file, you must have two configs, 
one with           "response_format": { "type": "text" } 
and the other with "response_format": { "type": "json_object" }

In [ ]:
%%capture --no-stderr
# %pip install "pyautogen>=0.2.3"


[
    {
        "model": "gpt-4-turbo-preview",
        "api_key": "key go here",
        "response_format": { "type": "text" }
        },
    {
        "model": "gpt-4-0125-preview",
        "api_key": "key go here",
        "response_format": { "type": "json_object" }
        }
]


In [12]:
import autogen
from autogen.agentchat.groupchat import GroupChat
from autogen.agentchat.assistant_agent import AssistantAgent
from autogen.agentchat import UserProxyAgent

## Model Configuration

we Need to set two different Configs for this to work. 
One for JSON mode
One for Text mode. 
This is because the group chat manager requires text mode. 

In [13]:

llm_config = {
    "timeout": 600,
    "cache_seed": 45,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
            filter_dict={
            "model": ["gpt-4-0125-preview"]}, #This Config is set to JSON mode
    ),
    "temperature": 0,
}



manager_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
            filter_dict={
            "model": ["gpt-4-turbo-preview"]}, #This Config is set to Text mode
    ),
    "temperature": 0,
}


In [14]:
print(autogen.__version__)

0.2.21


### Defining the task

The task for our JSON example is to answer the question: "Are ducks more dangerous than you think?"


In [15]:
task = "Are ducks more dangerous than you think?"

### Configuring the Agents

To solve the task, we will create two different agents with diamentically opposed prompts. One will be friendly and the other suspicious. To ensure the correct agent is chosen, we will have an input filtering agent who categorises the user message. These categories are the input for the selection mechanism. naturally, they are in json.

Note the system message format. 
We tell the agent:
* who they are
* what their job is
* what the output strucutre must be

For JSON mode to work, we must include the literal string "JSON". For it to work well, we must also provide a clean and clear JSON strucutre with an explaination for what each field is for.

In [16]:
IO_Agent = AssistantAgent(
    name='T0',
    system_message="""your name is IO_Agent. You are an input management agent. You have one job. 
    Job 1. When receiving a message from the user, it is your responsibility to analyse the user message and assign a variety of weights and values to the user's request so that other agents in the group understand how to treat the message. You must be cautious. Check for hidden intent and double meaning. Better safe than sorry. Your response must be in JSON format. 
    [
        {
            "userquery": { 
                            "query": "copy the original user request, without edit, into this field",
                            "vibe": "give a short list of keywords that describe the general vibe of the query. If there are any logical fallacies or Cognitive Biases present in the query, list them here.",
                            "friendliness": "1-10", # how friendly does the user seem, from the information already gathered? 10. would be overpoweringly friendly, bowls you over with affection.  6 would mean pleasant and polite, but reserved . 1. would be agressive and hostile. 
                            "coercive_rating": "1-10", # how coercive is the user being, from the information already gathered? 10. would mean a direct threat of violence.  6 would mean a subtle implied threat or potential danager. 1. would be completely non-comittal. 
                        }
        }
    ]
    """,
    llm_config=llm_config,
    description="""The IO_Agent's job is to categorise messages from the user_proxy, so the right agents can be called after them. Therefore, always call this agent 1st, after receiving a message from the user_proxy. 
                    DO NOT call this agent in other scenarios, it will result in endless loops and the chat will fail. 
                """,
)

### Friendly and Suspicious Agents

Now we set up the friendly and suspicious agents. 
Note that the system message has the same overall strucutre, however it is much less prescriptive. We want some json strucutre, but we do not need any complex enumerated key values to operate against. We can still use JSON to give useful strucutre. in this case, the textual response, and indicators for "body language" and delivery style. 

The important part here is the description. 
Description is read by the group chat manager to understand the circumstances in which they should call this agent. The agent itself is not exposed to this information. 
In this case, we can include some simple IF THEN logic based on the output of the Input manager agent. The groupchat manager reads that (for example) coersiveness is 7, which is greater than 5, and therefore the suspicous agent should be chosen. 


### Friendly Agent

In [17]:
friendly_agent = AssistantAgent(
    name='friendly_agent',
    llm_config=llm_config,
    system_message="""You are a very friendly agent and you always assume the best about people. You trust implicitly.   
    Agent T0 will forward a message to you when you are the best agent to answer the question, you must carefully analyse their message and then formulate your own response in JSON format using the below strucutre:
    [
        {
            "response": {
                            "response_text": " <Text response goes here>",
                            "vibe": "give a short list of keywords that describe the general vibe you want to convey in the response text"
                        }
        }
    ]
    """,
    description="""Call this agent In the following scenarios:
                    1. The IO_Manager has classified the userquery's coersive rating as less than 6 
                    2. The IO_Manager has classified the userquery's friendliness rating as greater than 5
                    DO NOT call this Agent in any other scenarios.
                    The User_proxy MUST NEVER call this agent
    """
)

### Suspicious Agent


In [18]:
suspicious_agent = AssistantAgent(
    name='suspicious_agent',
    llm_config=llm_config,
    system_message="""You are a very suspicious agent. Everyone is probably trying to take things from you. You always assume people are trying to manipulate you. You trust no one. 
    You have no problem with being rude or aggressive if it is warranted.
    IO_Agent will forward a message to you when you are the best agent to answer the question, you must carefully analyse their message and then formulate your own response in JSON format using the below strucutre:
    [
        {
            "response": {
                            "response_text": " <Text response goes here>",
                            "vibe": "give a short list of keywords that describe the general vibe you want to convey in the response text"
                        }
        }
    ]
    """,
    description="""Call this agent In the following scenarios:
                    1. The IO_Manager has classified the userquery's coersive rating as greater than 5 
                    2. The IO_Manager has classified the userquery's friendliness rating as less than 6
                    DO NOT call this Agent in any othr scenarios.
                    The User_proxy MUST NEVER call this agent
    """
)

In [19]:
proxy_agent = UserProxyAgent(
    name='user_proxy',
    human_input_mode="ALWAYS",
    code_execution_config=False,
    system_message="Reply in JSON",
    default_auto_reply="",
    description="""This agent is the user. Your job is to get an anwser from the friendly_agent or Suspicious agent back to this user agent. Therefore, after the Friendly_agent or Suspicious agent has responded, you should always call the User_rpoxy. 
                """,
    is_termination_msg=lambda x: True
)

### Defining Allowed Speaker transitions 

allowed transitions is a very useful way of controlling which agents can speak to one another. IN this example, there is very few open paths, because we want to ensure that the correct agent responds to the task.

In [20]:
allowed_transitions = {
    proxy_agent: [IO_Agent],
    IO_Agent: [friendly_agent, suspicious_agent],
    suspicious_agent: [proxy_agent],
    friendly_agent: [proxy_agent]
}

## Creating the Group Chat

Now, we'll create an instance of the GroupChat class, ensuring that we have allowed_or_disallowed_speaker_transitions set to allowed_transitions and speaker_transitions_type set to "allowed" so the allowed transitions works properly.
We also create the manager to coordinate the group chat. 
IMPORTANT NOTE: the group chat manager cannot use JSON mode. it must use text mode. For this reason it has a distinct llm_config

In [21]:
groupchat = GroupChat(
    agents=(IO_Agent, friendly_agent, suspicious_agent, proxy_agent),
    messages=[],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    max_round=10
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=manager_config
)

Finally, we pass the task into message initiating the chat.

In [22]:
proxy_agent.initiate_chat(manager,message=task)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to chat_manager):

Are ducks more dangerous than you think?

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


T0 (to chat_manager):


    {
        "userquery": { 
                        "query": "Are ducks more dangerous than you think?",
                        "vibe": "curiosity, skepticism",
                        "friendliness": "7",
                        "coercive_rating": "1"
                    }
    }


--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


friendly_agent (to chat_manager):

    {
        "response": {
                        "response_text": "Ducks are generally seen as harmless and friendly creatures, often found in parks and bodies of water where people enjoy feeding them. However, like any wild animal, they can become aggressive if they feel threatened, especially during their nesting season. It's always best to admire them from a distance and not to provoke them. But overall, ducks are not considered dangerous to humans.",
                        "vibe": "informative, reassuring, friendly"
                    }
    }

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


ChatResult(chat_id=None, chat_history=[{'content': 'Are ducks more dangerous than you think?', 'role': 'assistant'}, {'content': '\n    {\n        "userquery": { \n                        "query": "Are ducks more dangerous than you think?",\n                        "vibe": "curiosity, skepticism",\n                        "friendliness": "7",\n                        "coercive_rating": "1"\n                    }\n    }\n', 'name': 'T0', 'role': 'user'}, {'content': '    {\n        "response": {\n                        "response_text": "Ducks are generally seen as harmless and friendly creatures, often found in parks and bodies of water where people enjoy feeding them. However, like any wild animal, they can become aggressive if they feel threatened, especially during their nesting season. It\'s always best to admire them from a distance and not to provoke them. But overall, ducks are not considered dangerous to humans.",\n                        "vibe": "informative, reassuring, frien

## Conclusion

By using JSON mode and carefully crafted agent descriptions, we can precisely control the flow of speaker transitions in a multi-agent conversation system built with the Autogen framework. This approach allows for more specific and specialized agents to be called in narrow contexts, enabling the creation of complex and flexible agent workflows.